In [1]:
import requests
import pandas as pd
import json
from dotenv import load_dotenv

from functools import reduce

load_dotenv()
league_id = os.getenv('league_id')
url = "https://fantasy.espn.com/apis/v3/games/ffl/seasons/2020/segments/0/leagues/{}".format(league_id)
history_url = "https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/{}".format(league_id)
cookies = {
        "swid": os.getenv('swid'),
        "espn_s2": os.getenv('espn_s2')
    }


In [2]:
def add_year(game, season_id):
    game["year"] = season_id
    return game

def get_schedule(season_id):
    hist_req = requests.get(history_url,
        cookies=cookies,
        params={"view": "mMatchup", "seasonId": season_id,})
    hist_list = list(map(lambda x: add_year(x, season_id), hist_req.json()[0]["schedule"]))
    return hist_list

games_r = requests.get(url,
params={"view": "mMatchup"},
    cookies=cookies)
schedule = list(map(lambda x: add_year(x, 2020), games_r.json()["schedule"]))

In [3]:
prev = games_r.json()["status"]["previousSeasons"]
prev_games = list(map(get_schedule, prev))

In [4]:
all_games = schedule + reduce(list.__add__, prev_games)

games_df = pd.DataFrame([[game["matchupPeriodId"], 
game["home"]["teamId"], game["home"]["totalPoints"],
game["away"]["teamId"], game["away"]["totalPoints"], game["year"]] for game in all_games],
    columns=['Week', 'Team1', 'Score1', 'Team2', 'Score2', "year"])
games_df.head()

,Week,Team1,Score1,Team2,Score2,year
0,1,1,139.0,13,109.6,2020
1,1,12,146.8,9,122.8,2020
2,1,16,129.4,3,129.5,2020
3,1,10,142.3,15,167.0,2020
4,1,7,145.3,17,110.7,2020


In [5]:
game_margin_df = games_df.assign(Margin1 = games_df["Score1"] - games_df["Score2"],
    Margin2 = games_df["Score2"] - games_df["Score1"])
game_margin_df.head()

,Week,Team1,Score1,Team2,Score2,year,Margin1,Margin2
0,1,1,139.0,13,109.6,2020,29.4,-29.4
1,1,12,146.8,9,122.8,2020,24.0,-24.0
2,1,16,129.4,3,129.5,2020,-0.1,0.1
3,1,10,142.3,15,167.0,2020,-24.7,24.7
4,1,7,145.3,17,110.7,2020,34.6,-34.6


In [6]:
g_flattened = (game_margin_df[['Week', 'Team1', 'Margin1', 'Score1', 'Score2', 'year']]
    .rename(columns={'Team1': 'Team', 'Score1': 'Score', 'Margin1': 'Margin', 'Score2': 'OppScore'})
    .append(game_margin_df[['Week', 'Team2', 'Margin2', 'Score2', 'Score1', 'year']]
    .rename(columns={'Team2': 'Team', 'Score2': 'Score', 'Margin2': 'Margin', 'Score1': 'OppScore'})
    ))

g_flattened.to_csv("games.csv")